In [1]:
# need to create a Python environment to install R and R studio using conda. Activate the environment and then open this Jupyter notebook.
import pandas as pd
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import numpy as np
import math
import os

## Input data prep

### Populating color matrix

In [11]:
# color code by phyla
otu_taxon = pd.read_csv("input/OTU_taxon.csv")
otu_network_color = otu_taxon.drop(["OTUs", "Class", "Order", "Family", "Genus", "Species"], axis=1)
otu_network_color["id"] = otu_network_color.index
otu_network_color = otu_network_color[['id', 'Phylum']]
otu_network_color["id"] += 1
otu_network_color.to_excel("input/otu_network_color_phyla.xlsx", index=False)

In [12]:
# color code by abundance
otu_network_color = pd.read_csv("input/abun_rare.csv")
otu_network_color["id"] = otu_network_color.index
otu_network_color = otu_network_color[['id', 'Ecotype']]
otu_network_color["id"] += 1
otu_network_color.to_excel("input/otu_network_color_abundance.xlsx", index=False)

In [13]:
# color code by niche breadth
otu_network_color = pd.read_csv("input/gen_spe.csv")
otu_network_color["id"] = otu_network_color.index
otu_network_color = otu_network_color[['id', 'Ecotype']]
otu_network_color["id"] += 1
otu_network_color.to_excel("input/otu_network_color_breadth.xlsx", index=False)

### Processing all OTUs

In [52]:
otu_relative = pd.read_csv("input/OTU_rar_relative%.csv", index_col=0)
otu_relative = otu_relative.transpose()
# Save the transposed DataFrame to a new CSV, without the index as the first column
otu_relative.to_csv("input/transposed_OTU_rar_relative.csv", index=True)

### Processing a subset of OTUs for testing

In [16]:
# creating subset of data for testing
transposed_OTU_rar_relative = pd.read_csv("input/transposed_OTU_rar_relative.csv")
subset = transposed_OTU_rar_relative.iloc[:100, :100]
subset.to_csv("input/subset_transposed_OTU_rar_relative.csv", index=False)

# Populating color matrix for testing
otu_network_color = pd.read_excel("input/otu_network_color_phyla.xlsx")
otu_network_color_subset = otu_network_color.iloc[:99,:]
otu_network_color_subset.to_excel("input/subset_otu_network_color.xlsx", index=False)

## OTU network metric function

In [46]:
def network_metrics(result, network_color_file):
    network_color_list = network_color_file["id"].tolist()

    lines = result.split('\n')
    edge_density = float(lines[0][len('[1] '):])

    degree = {}
    closeness = {}
    betweenness = {}
    current_stage = ''
    lines = lines[1:]
    a = iter(lines)
    for line1, line2 in zip(a, a):
        if 'degree' in line1 or 'closeness' in line1 or 'betweenness' in line1:
            current_stage = line1.strip()  # Also stripping possible whitespace
            continue
        keys = line1.split()
        vals = line2.split()
        for k, v in zip(keys, vals):
            if 'degree' in current_stage:
                degree[k] = int(v)
            elif 'closeness' in current_stage:
                closeness[k] = float(v)
            elif 'betweenness' in current_stage:
                betweenness[k] = float(v)

    # Convert keys to 'OTU' format after sorting them numerically
    all_keys = sorted(set(degree.keys()).union(closeness.keys(), betweenness.keys()), key=int)
    all_keys = ["OTU" + k for k in all_keys]  # Add 'OTU' prefix after sorting

    df_data = {
        'id': network_color_list,
        'degree': [degree.get(k[3:], 0) for k in all_keys],  # Strip 'OTU' before accessing the value
        'closeness': [closeness.get(k[3:], 0) for k in all_keys],  # Strip 'OTU' before accessing the value
        'betweenness': [betweenness.get(k[3:], 0) for k in all_keys]  # Strip 'OTU' before accessing the value
    }
    df = pd.DataFrame(df_data)
    return edge_density, df

### Subset for testing

In [4]:
subset_otu_network_color = "input/subset_otu_network_color.xlsx"
subset_otu_network_color_df = pd.read_excel("input/subset_otu_network_color.xlsx")
subset_otu_rar_relative = "input/subset_transposed_OTU_rar_relative.csv"

In [44]:
# Generate network metrics
subset_otu_network_metrics = subprocess.check_output(['Rscript', 'SpiecEasi_stage_python_phyla.R', subset_otu_rar_relative, subset_otu_network_color],
                                 universal_newlines=True, stderr=subprocess.DEVNULL)

In [47]:
edge_density, df = network_metrics(subset_otu_network_metrics, subset_otu_network_color_df)
print("Edge_density: ", edge_density)

Edge_density:  0.01752216


In [17]:
# Plot network
subprocess.check_output(['Rscript', 'SpiecEasi_stage_network_phyla.R',
                         subset_otu_rar_relative, subset_otu_network_color,
                         'output/network_SpiecEasi_subset.pdf','output/edge_weight_subset.csv'], universal_newlines=True, stderr=subprocess.DEVNULL)

'null device \n          1 \n'

### All OTUs

In [24]:
otu_network_color_phyla = "input/otu_network_color_phyla.xlsx"
otu_network_color_abundance = "input/otu_network_color_abundance.xlsx"
otu_network_color_breadth= "input/otu_network_color_breadth.xlsx"

otu_network_color_df = pd.read_excel("input/otu_network_color_phyla.xlsx")

otu_rar_relative = "input/transposed_OTU_rar_relative.csv"

In [60]:
# Generate network metrics
otu_network_metrics = subprocess.check_output(['Rscript', 'SpiecEasi_stage_python_phyla.R', otu_rar_relative, otu_network_color],
                                 universal_newlines=True, stderr=subprocess.DEVNULL)

In [61]:
edge_density, df = network_metrics(otu_network_metrics, otu_network_color_df)
print("Edge_density: ", edge_density)

Edge_density:  0.01666492


In [68]:
edge_density_df = pd.DataFrame(data={'edge density': [edge_density]})
edge_density_df.to_csv("output/edge_density_all_OTUs.csv", index=False)

In [69]:
df.to_csv("output/network_metrics_all_OTUs.csv", index = False)

In [19]:
# Plot network color code by phyla

subprocess.check_output(['Rscript', 'SpiecEasi_stage_network_phyla.R',
                         otu_rar_relative, otu_network_color_phyla,
                         'output/network_SpiecEasi_all_OTUs_phyla.pdf','output/edge_weight_all_OTUs.csv'], universal_newlines=True, stderr=subprocess.DEVNULL)

'null device \n          1 \n'

In [21]:
# Plot network color code by abundance

subprocess.check_output(['Rscript', 'SpiecEasi_stage_network_ecotypes.R',
                         otu_rar_relative, otu_network_color_abundance,
                         'output/network_SpiecEasi_all_OTUs_abundance.pdf','output/edge_weight_all_OTUs.csv'], universal_newlines=True, stderr=subprocess.DEVNULL)

'null device \n          1 \n'

In [25]:
# Plot network color code by niche breadth

subprocess.check_output(['Rscript', 'SpiecEasi_stage_network_ecotypes.R',
                         otu_rar_relative, otu_network_color_breadth,
                         'output/network_SpiecEasi_all_OTUs_breadth.pdf','output/edge_weight_all_OTUs.csv'], universal_newlines=True, stderr=subprocess.DEVNULL)

'null device \n          1 \n'

### Filtering OUTs to improve network visualization if needed
Since the network plot is cluttered. Reducing the number of OTU for analysis by the following two step process. 1. Drop OTUs which are present in X% of samples 2. Drop OTUs which have spearman correlation less than y with all other OTUs

In [8]:
# df = pd.read_csv('input/transposed_OTU_rar_relative.csv', index_col=0)

# df.replace(0, np.nan, inplace=True)

# # Step 1
# threshold = math.floor(0.05*618) #
# columns_to_drop_1 = []
# for col in df.columns:
#     # Count non-zero entries (NaN values are automatically not counted)
#     non_zero_count = df[col].count()

#     # If the count of non-zero entries is less than the threshold, mark the column for dropping
#     if non_zero_count < threshold:
#         columns_to_drop_1.append(col)
# print("Fol threshold: ", threshold,", the number of cols to be dropped is ", len(columns_to_drop_1))


# df = pd.read_csv('input/transposed_OTU_rar_relative.csv', index_col=0)
# df = df.drop(columns=columns_to_drop_1)

# # Step 2
# correlation_matrix = df.corr(method='spearman')

# threshold = 0.4
# tolerance_list = [0] #number of allowable high correlations

# for tolerance in tolerance_list:
#     columns_to_drop_2 = []
#     for col in correlation_matrix.columns:
#         high_corr_count = (correlation_matrix[col].drop(col).abs() > threshold).sum()
#         if high_corr_count <= tolerance:
#             columns_to_drop_2.append(col)

#     print("Fol tolerance: ", tolerance,", the number of cols to be dropped is ", len(columns_to_drop_2))

# df = df.drop(columns=columns_to_drop_2)
# df.to_csv("input/less_than_5_and_4_OTU_data.csv")

Fol threshold:  30 , the number of cols to be dropped is  1974
Fol tolerance:  0 , the number of cols to be dropped is  405


In [9]:
# # dropping the same OTU from the color csv file
# columns_to_drop = columns_to_drop_1 + columns_to_drop_2
# ids_to_drop = [int(col[4:]) for col in columns_to_drop]
# otu_network_color = pd.read_excel("input/otu_network_color_phyla.xlsx")
# df_color_filtered = otu_network_color[~otu_network_color['id'].isin(ids_to_drop)]
# df_color_filtered.to_excel("input/less_than_5_and_4_otu_network_color.xlsx", index=False)

In [10]:
# # load input files for network construction
# filtered_otu_network_color = "input/less_than_5_and_4_otu_network_color.xlsx"
# filtered_otu_rar_relative = "input/less_than_5_and_4_OTU_data.csv"

In [13]:
# # Plot network

# subprocess.check_output(['Rscript', 'SpiecEasi_stage_network_phyla.R',
#                          filtered_otu_rar_relative, filtered_otu_network_color,
#                          'output/network_SpiecEasi_filtered_OTUs.pdf','output/edge_weight_filtered_OTUs.csv'], universal_newlines=True, stderr=subprocess.DEVNULL)

'null device \n          1 \n'

## Network for ecosystems

In [49]:
ecosystem = pd.read_csv("input/ecosystem_modified.csv")

In [50]:
transposed_OTU_rar_relative = pd.read_csv("input/transposed_OTU_rar_relative.csv")

In [51]:
# merging to get Ecosystem class for all samples
merged_data = pd.merge(ecosystem, transposed_OTU_rar_relative, left_on="Sample ID ", right_on="Unnamed: 0")

In [52]:
# Create separate dataframes for each ecosystem class
ecosystem_classes = merged_data['Ecosystem_recode'].unique()
ecosystems_dfs = {ecosystem: merged_data[merged_data['Ecosystem_recode'] == ecosystem].drop(columns=['Ecosystem_recode', 'Sample ID ']) for ecosystem in ecosystem_classes}

In [53]:
# Show initial sample counts for each class
initial_counts = {ecosystem: len(df) for ecosystem, df in ecosystems_dfs.items()}
print("Initial Sample Counts:", initial_counts)

Initial Sample Counts: {'ForestWoodland': 298, 'Barren': 26, 'Wetland': 56, 'Unknown': 29, 'Shrubland': 44, 'Herbaceous': 134, 'SteppeSavanna': 31}


In [54]:
# Find minimum sample count across all dataframes
min_sample_count = min(initial_counts.values())
min_sample_count

26

In [55]:
# Randomly drop samples to make counts equal to min count among ecosystem classes.
equalized_dfs = {}
for ecosystem, df in ecosystems_dfs.items():
    if len(df) > min_sample_count:
        equalized_dfs[ecosystem] = df.sample(n=min_sample_count, random_state=42)
    else:
        equalized_dfs[ecosystem] = df

In [56]:
# Remove OTUs with zero presence in all samples for each DataFrame
final_dfs = {}
final_otu_counts = {}
for ecosystem, df in equalized_dfs.items():
    df = df.loc[:, (df != 0).any(axis=0)]
    final_dfs[ecosystem] = df
    final_otu_counts[ecosystem] = df.shape[1]

In [57]:
# Display final DataFrame structures and sample counts
final_counts = {ecosystem: len(df) for ecosystem, df in final_dfs.items()}
print("Final Sample Counts:", final_counts)
print("Final OTU Counts:", final_otu_counts)

Final Sample Counts: {'ForestWoodland': 26, 'Barren': 26, 'Wetland': 26, 'Unknown': 26, 'Shrubland': 26, 'Herbaceous': 26, 'SteppeSavanna': 26}
Final OTU Counts: {'ForestWoodland': 1825, 'Barren': 1948, 'Wetland': 1962, 'Unknown': 1936, 'Shrubland': 1378, 'Herbaceous': 1685, 'SteppeSavanna': 1784}


In [58]:
# Since the number of OTUs after removing the OTUs with zero presence in all samples is still high
# Remove OTUs whihc are present in less than or equal to threshold number of samples
final_dfs = {}
otu_presence_counts = {}
thresholds = [0, 1, 2, 3, 4, 5]
for ecosystem, df in equalized_dfs.items():
    df = df.loc[:, (df != 0).any(axis=0)]
    final_dfs[ecosystem] = df
    num_samples = len(df)
    otu_presence_counts[ecosystem] = {}
    for threshold in thresholds:
        count = (df != 0).sum(axis=0) > threshold
        otu_presence_counts[ecosystem][f"{int(threshold)} count"] = count.sum()

# Display final DataFrame structures and OTU presence counts
print("OTU Presence with non zero Counts:")
for ecosystem, counts in otu_presence_counts.items():
    print(f"{ecosystem}: {counts}")

# The final counts indicate that a threshold value of 3 gives a manageable OTU count.

OTU Presence with non zero Counts:
ForestWoodland: {'0 count': 1825, '1 count': 1068, '2 count': 769, '3 count': 593, '4 count': 484, '5 count': 403}
Barren: {'0 count': 1948, '1 count': 1162, '2 count': 754, '3 count': 563, '4 count': 425, '5 count': 322}
Wetland: {'0 count': 1962, '1 count': 1190, '2 count': 808, '3 count': 599, '4 count': 453, '5 count': 348}
Unknown: {'0 count': 1936, '1 count': 1155, '2 count': 824, '3 count': 615, '4 count': 487, '5 count': 396}
Shrubland: {'0 count': 1378, '1 count': 915, '2 count': 656, '3 count': 498, '4 count': 398, '5 count': 320}
Herbaceous: {'0 count': 1685, '1 count': 1044, '2 count': 716, '3 count': 545, '4 count': 438, '5 count': 387}
SteppeSavanna: {'0 count': 1784, '1 count': 1059, '2 count': 737, '3 count': 551, '4 count': 452, '5 count': 367}


In [59]:
final_dfs = {}
otu_presence_counts = {}
ecosystem_directories = []
threshold = 1  # Set the threshold of interest
for ecosystem, df in equalized_dfs.items():
    if ecosystem == "Unknown":
        continue
    # Filter out OTUs with zero presence across all samples
    df = df.loc[:, (df != 0).any(axis=0)]
    final_dfs[ecosystem] = df

    # Calculate the number of samples in which each OTU is present more than the threshold
    presence_count = (df != 0).sum(axis=0) > threshold
    otu_presence_counts[ecosystem] = presence_count.sum()

    # Check if directory exists for the ecosystem, if not create it
    directory = f'output/{ecosystem}/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    ecosystem_directories.append(directory)
    # Save the filtered DataFrame to a CSV file in the newly created or existing directory
    filtered_df = df.loc[:, (df != 0).sum(axis=0) > threshold]
    filtered_df.to_csv(f'{directory}filtered_otus.csv', index = False)

# Display the count of OTUs that meet the criteria for the threshold
print("OTU Presence with non-zero counts above the threshold of 1:")
for ecosystem, count in otu_presence_counts.items():
    print(f"{ecosystem}: {count}")

OTU Presence with non-zero counts above the threshold of 1:
ForestWoodland: 1068
Barren: 1162
Wetland: 1190
Shrubland: 915
Herbaceous: 1044
SteppeSavanna: 1059


In [60]:
# fixing network color files
for directory in ecosystem_directories:
    otu_network_color = pd.read_excel("input/otu_network_color_phyla.xlsx")

    # Read the current filtered OTU file
    df_otu = pd.read_csv(f"{directory}filtered_otus.csv")

    # Get OTU IDs from column names
    otus = df_otu.columns.tolist()[1:] # dropping the first because it is col name for Sample id; not an otu
    otu_ids = [int(otu.split('_')[1]) for otu in otus]

    # Filter the color data to keep only rows where the 'id' matches the OTUs in the current file
    df_color_filtered = otu_network_color[otu_network_color['id'].isin(otu_ids)]

    # Save the filtered color data to an Excel file
    df_color_filtered.to_excel(f'{directory}filtered_network_color.xlsx', index=False)

In [61]:
# this function will find the network metrics using SpiecEasi_stage_python.R fie. This function uses OS subprocess to call the R script. The R script must be in the same folder as this file
for directory in ecosystem_directories:
    input_csv_file_name = f'{directory}filtered_otus.csv'
    color_file_name = f'{directory}filtered_network_color.xlsx'
    pdf_file_name = f'{directory}network_SpiecEasi.pdf'
    edge_weight_file_name = f'{directory}edge_weight.csv'

    color_file = pd.read_excel(color_file_name)

    result_tb_obs_otu_full = subprocess.check_output(['Rscript', 'SpiecEasi_stage_python_phyla.R', input_csv_file_name, color_file_name], universal_newlines=True, stderr=subprocess.DEVNULL)
    edge_density, df = network_metrics(result_tb_obs_otu_full, color_file)
    print(f"For the data in {input_csv_file_name} file")
    print("Edge_density: ", edge_density)
    print("Degree, closeness, betweeness results are as follows: ")
    print(df)

    edge_density_df = pd.DataFrame([edge_density], columns=["edge density"])
    df.to_csv(f'{directory}network_metrics.csv', index=False)
    edge_density_df.to_csv(f'{directory}edge_density.csv', index = False)
    subprocess.check_output(['Rscript', 'SpiecEasi_stage_network_phyla.R', input_csv_file_name, color_file_name, pdf_file_name, edge_weight_file_name], universal_newlines=True, stderr=subprocess.DEVNULL)

For the data in output/ForestWoodland/filtered_otus.csv file
Edge_density:  0.02545933
Degree, closeness, betweeness results are as follows: 
        id  degree  closeness  betweenness
0        2      32   0.000004        993.0
1        3      20   0.000004       1522.0
2        5      11   0.000003        842.0
3        7      34   0.000005       1525.0
4       10      29   0.000004       1045.0
...    ...     ...        ...          ...
1062  3129      44   0.000004         59.0
1063  3132      40   0.000005        760.0
1064  3145      30   0.000004        832.0
1065  3155      30   0.000004       2047.0
1066  3158      48   0.000004        140.0

[1067 rows x 4 columns]
For the data in output/Barren/filtered_otus.csv file
Edge_density:  0.02248211
Degree, closeness, betweeness results are as follows: 
        id  degree  closeness  betweenness
0        2      40   0.000004       1014.0
1        3     108   0.000003          5.0
2        5      17   0.000003        833.0
3        6 